In [1]:
import numpy as onp
import jaxlie
from jax import numpy as np
import jax
import time

In [2]:
# Set to CPU
# Comment out to use GPU/TPU
# jax.config.update('jax_platform_name', 'cpu')

In [3]:
jax.lib.xla_bridge.get_backend().platform

'gpu'

In [4]:
def time_func(f, key, calls):
    start = time.perf_counter()
    for _ in range(calls):
        f(key)
        _, key = jax.random.split(key)
    end = time.perf_counter()
    return (end - start) / calls

In [5]:
key = jax.random.PRNGKey(42)

for N in reversed([1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]):
    N = int(N)
    mat_size = 7

    A = jax.random.normal(key, (N, mat_size, mat_size))
    _, key = jax.random.split(key)
    B = jax.random.normal(key, (N, mat_size, mat_size))
    _, key = jax.random.split(key)

    np.matmul(A, B)
    
    def matmul(key):
        A_new = A.at[0, 0, 0].set(jax.random.normal(key))
        return np.matmul(A_new, B)
                  
    t = time_func(matmul, key, 10)    
    
    def notmul(key):
        A_new = A.at[0, 0, 0].set(jax.random.normal(key))
        return A_new
                  
    _, key = jax.random.split(key)
    t2 = time_func(notmul, key, 10)
    
    print(f"{N:>10}   {t:10.5} {t2:10.5} {t - t2:10.5} {(t - t2) / N:10.5}")

  10000000     0.085016  0.0093498   0.075667 7.5667e-09
   1000000     0.012949  0.0027018   0.010247 1.0247e-08
    100000      0.00903  0.0021562  0.0068738 6.8738e-08
     10000    0.0089667  0.0021558  0.0068109 6.8109e-07
      1000    0.0088506  0.0021513  0.0066993 6.6993e-06
       100    0.0088216  0.0021411  0.0066804 6.6804e-05
        10    0.0079862  0.0021016  0.0058846 0.00058846
